In [1]:
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
parent_dir = os.path.dirname(os.getcwd())
data_path = os.path.join(parent_dir, 'data', 'dataset')

In [3]:
dataset = pd.read_csv(os.path.join(data_path, 'dataset.csv'))
print(dataset.shape)

(1312248, 15)


In [4]:
champs = dataset.CHAMPION.unique().tolist()
roles = dataset.ROLE.unique().tolist()
print("Total champs: ", len(champs))
print("Total roles: ", len(roles))

Total champs:  156
Total roles:  5


In [5]:
# no of campion role combinations possible. Max possible is 780 = 5x156
extracted = dataset.copy()
extracted['champ_role'] = extracted['CHAMPION'].str.cat(extracted[['ROLE']], sep='_')
extracted.champ_role.nunique()

772

In [6]:
# champ in role count distribution.
# Only keep those combinations which have atleast 15 records
counts = extracted.champ_role.value_counts().reset_index(name='count')
counts = counts.query('count > 15')
counts

,index,count
0,Ezreal_bottom,35172
1,Kaisa_bottom,28125
2,Thresh_utility,26293
3,LeeSin_jungle,23512
4,Lulu_utility,20272
...,...,...
560,Leblanc_bottom,16
561,TwistedFate_jungle,16
562,Camille_bottom,16
563,Volibear_utility,16


In [7]:
extracted = extracted[extracted.champ_role.isin(counts['index'])]
extracted = extracted[['CHAMPION', 'ROLE', 'champ_role', 'KILLS', 'ASSISTS', 'DEATHS']]
extracted

,CHAMPION,ROLE,champ_role,KILLS,ASSISTS,DEATHS
0,Kassadin,top,Kassadin_top,14,5,7
1,Fiora,top,Fiora_top,3,13,6
2,Yone,middle,Yone_middle,7,3,6
3,Corki,middle,Corki_middle,4,15,7
4,Kaisa,bottom,Kaisa_bottom,5,10,12
...,...,...,...,...,...,...
1312243,Alistar,utility,Alistar_utility,5,16,6
1312244,LeeSin,jungle,LeeSin_jungle,2,5,7
1312245,FiddleSticks,jungle,FiddleSticks_jungle,5,16,5
1312246,Shen,top,Shen_top,1,1,3


In [8]:
c_r_kda = extracted.groupby(['CHAMPION', 'ROLE']).apply(lambda x: (x['KILLS'].sum() + x['ASSISTS'].sum()) / (x['DEATHS'].sum())).reset_index(name = 'kda')

In [9]:
c_r_kda

,CHAMPION,ROLE,kda
0,Aatrox,bottom,2.348837
1,Aatrox,jungle,2.224719
2,Aatrox,middle,2.337035
3,Aatrox,top,2.046735
4,Ahri,bottom,2.498457
...,...,...,...
560,Zoe,top,2.452941
561,Zoe,utility,1.945255
562,Zyra,bottom,2.552058
563,Zyra,middle,2.238857


In [10]:
c_r_kda.to_csv(os.path.join(data_path, 'champ_role_kda.csv'), index = False)